## 0. Описание ноутбука

   В данном ноутбуке будет проводится отбор и обучение моделей, на основании датасета подготовленного ранее.

   После выбора итоговой модели необходимо будет сделать предсказание (submission) для соревнования на Kaggle.
   
   При подготовке предсказание необходимо помнить, что целевая переменная в датасете логарифмирована (не забыть проэкспонировать), а так-же что необходимо провести корректировку цен из-за изменения экономической обстановки (т.к. между парсингом тестовых и тренировочных данных прошло больше года).
   
   Процесс парсинга данных и EDA представлены в двух "предыдущих" ноутбуках, сохраненных в этом-же репозитории.
   
   Т.к. обучение моделей (а особенно выбор модели с помощью LazyPredict или подбор гиперпараметров модели с помощью RandomizedSearchCV) это очень долгий процесс, соответствующие ячейки закомментированы, что-бы не перезапускать их каждый раз.
   
   Результаты выполнения всех "долгих" ячеек представлены под ними в текстовом виде для логарифмированной и НЕ логарифмированной целевой переменной.

## 1. Импорт библиотек и настройка параметров

Подключим необходимые бибилиотеки, пропишем пути для файловых операций и зададим системные параметры.

In [ ]:
# При необходимости доставляем библиотеки:

#!pip install --user CatBoostRegressor 
#!pip install --user lazypredict

In [ ]:
# Импортируем библиотеки:

import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.ensemble import StackingRegressor
from xgboost import XGBRegressor

import lazypredict
from lazypredict.Supervised import LazyRegressor

from catboost import CatBoostRegressor

In [2]:
# Увеличиваем число отображаемых строк и столбцов в pandas:

pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

In [3]:
# Задаем дирректорию для работы с файлами:

# При выполнении на kaggle раскомментировать этот блок:
# for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))
# PATH_to_file = '/kaggle/input/sf-dst-scoring/'

# При выполнении локально раскомментировать этот блок:
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
PATH_to_file = './data/'

./data/cars_dicts.txt
./data/submission_log.csv
./data/not_read_links.txt
./data/car_links.txt
./data/test.csv
./data/df_prepared.csv
./data/submission_norm.csv
./data/train.csv
./data/test_finished.csv
./data/pages_count_list.txt
./data/sample_submission.csv


In [4]:
# Фиксируем параметры для воспроизводимости экспериментов:

RANDOM_SEED = 42
VAL_SIZE   = 0.20
!pip freeze > requirements.txt

In [5]:
# Создадим функцию для вычисления итоговой метрики:

def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred - y_true) / y_true))

## 2. Подготовка данных

Готовим данные для моделирования на основе подготовленного на прошлых этапах датасета.

### 2.1 Загрузка датасета

In [6]:
# Загрузим подготовленный датасет:

df = pd.read_csv(PATH_to_file + 'df_prepared.csv')

print('Размер датасета: ', df.shape)
display(df.sample(5))

Размер датасета:  (126689, 38)


,body_type,brand,color,engine_power,fuel_type,mileage,model_name,doors_num,production_date,transmission,vendor,owners_count,docs,drive,steering,price,test,pop_color,pop_brand,elite_brand,pop_model,prod_annot,km_year,rarity,desc_digits,desc_len,options_count,airbag-passenger,aux,electro-mirrors,electro-window-back,esp,front-seats-heat,immo,mirrors-heat,other,ptf,usb
24638,9,22,7,170,3,350000,598,2,2002,0,0,2,0,1,0,0.00,1,0,1,0,1,2,18421,0,0,309,18,0.00,0.00,0.00,0.00,0.00,1.00,0.00,1.00,1.00,0.00,1.00
66304,10,10,13,170,3,308000,315,3,2001,1,0,0,0,2,0,12.85,0,1,1,1,1,2,15400,0,0,86,17,0.00,0.00,1.00,0.00,0.00,0.00,1.00,0.00,1.00,0.00,0.00
86369,5,18,13,90,3,128000,673,3,2016,1,0,0,1,1,0,13.69,0,1,1,0,1,4,25600,0,0,184,27,0.00,1.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
6655,0,23,1,210,3,182000,915,3,2007,0,0,0,1,0,0,0.00,1,1,0,0,1,1,13000,0,26,329,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
108795,9,22,1,110,3,39000,616,2,2019,1,0,1,1,1,0,13.84,0,1,1,0,1,5,19500,0,0,71,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### 2.2 Подготовка данных для обучения

In [7]:
# Разделим датасет на тестовый (для которого будем предсказывать цены)
# и тренировочный, на котором будем тренировать модель:

df_test = df[df.test == 1]

df_train = df[df.test == 0]

In [8]:
# Удалим признак test из обоих датасетов. Из тестового датасета так-же
# удалим признак price, который нам надо будет предсказать:

df_test = df_test.drop(columns = ['test', 'price'], axis = 1)

df_train = df_train.drop(columns = 'test', axis = 1)

In [9]:
# Выделяем целевую переменную из тренировочног датасета. Моделирование будем проводить
# по 2 раза, для значений полученных на прошлом этапе (т.е. логарифмированной целевой
# переменной) и для экспонированных (т.е. НЕ логарифмированной целевой переменной):

# Для НЕ логарифмированной целевой переменной используем:
#y = np.exp(df_train.price)

# Для логарифмированной целевой переменной используем:
y = df_train.price

X = df_train.drop(columns = 'price', axis = 1)

In [10]:
# Разделим данные для обучения следующим образом:

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    shuffle=True, 
                                                    random_state=RANDOM_SEED)

## 3. Моделирование

Теперь построим несколько моделей, для каждой из которых посчитаем нашу метрику для логарифмированной и НЕ логарифмированной переменных.

### 3.1 Наивная модель

In [12]:
# Эта модель будет предсказывать среднюю цену по мощности двигателя (engine_power). 
# C ней будем сравнивать другие модели.

In [11]:
tmp_train = X_train.copy()
tmp_train['price'] = y_train

In [12]:
# Находим median по экземплярам engine_power в трейне и размечаем тест
predict = X_test['engine_power'].map(tmp_train.groupby('engine_power')['price'].median())

#оцениваем точность
print(f"Точность наивной модели по метрике MAPE: {(mape(y_test, predict.values))*100:0.3f}%")

Точность наивной модели по метрике MAPE: 4.033%


    Для логарифмированной переменной: 
    Точность наивной модели по метрике MAPE: 4.033%

    Для НЕ логарифмированной переменной:
    Точность наивной модели по метрике MAPE: 75.610%

### 3.2 CatBoost

In [13]:
cat_boost = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
cat_boost.fit(X_train, y_train,
         #cat_features=cat_features_ids,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

cat_boost.save_model('catboost_single_model_baseline.model')

In [14]:
# оцениваем точность
predict = cat_boost.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.176%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.176%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 19.36%

### 3.3 LazyPredict

In [1]:
# Для выбора наилучших моделей воспользуемся библиотекой Lazypredict.
# LazyRegressor обучит сразу 42 различные модели на наших данных и выдаст 
# результаты для каждой из них. Задать нашу функцию в качестве дополнительной
# метрики не получится, так-что будем использовать близкую ей MAE.

In [ ]:
# ВНИМАНИЕ!!! ОЧЕНЬ ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Воспользуемся библиотекой Lazypredict для нахождения моделей, которые на дефолтных 
# параметрах отрабатывают лучше с нашими данными. 

#mae = mean_absolute_error

#lazy_regr = LazyRegressor(verbose=1,ignore_warnings=True,
#                          predictions=True, custom_metric=mae)

#models, predictions = lazy_regr.fit(X_train, X_test, y_train, y_test)

#print(models)

#### Результат прогона LazyPredict для НЕ логарифмированной целевой переменной:

                                   Adjusted R-Squared  R-Squared       RMSE  Time Taken  mean_absolute_error
    Model                                                                                                   
    XGBRegressor                                 0.92       0.92  481025.21        2.34            222166.40
    ExtraTreesRegressor                          0.91       0.91  490051.38       42.46            205402.77
    LGBMRegressor                                0.90       0.90  518023.81        0.55            242478.02
    HistGradientBoostingRegressor                0.90       0.90  525305.11        2.41            244044.29
    RandomForestRegressor                        0.90       0.90  532380.38       55.53            209929.92
    BaggingRegressor                             0.88       0.88  580306.54        5.86            223996.71
    GradientBoostingRegressor                    0.84       0.84  661127.32       12.88            316386.12
    DecisionTreeRegressor                        0.81       0.82  719311.80        1.03            285042.12
    ExtraTreeRegressor                           0.78       0.78  790122.15        0.52            330298.99
    KNeighborsRegressor                          0.76       0.76  811510.84       78.78            377830.23
    PoissonRegressor                             0.74       0.74  855349.53        0.32            394264.83
    LassoCV                                      0.60       0.60 1059892.24        1.72            588034.16
    BayesianRidge                                0.60       0.60 1059914.74        0.21            589188.59
    LassoLars                                    0.60       0.60 1059925.82        0.15            588963.96
    RidgeCV                                      0.60       0.60 1059933.54        0.24            589255.89
    LarsCV                                       0.60       0.60 1059936.46        0.68            589144.73
    LassoLarsCV                                  0.60       0.60 1059936.46        0.55            589144.73
    LassoLarsIC                                  0.60       0.60 1059945.43        0.18            589282.84
    Ridge                                        0.60       0.60 1059945.63        0.14            589298.90
    Lasso                                        0.60       0.60 1059946.89        2.63            589302.38
    Lars                                         0.60       0.60 1059946.99        0.32            589303.69
    TransformedTargetRegressor                   0.60       0.60 1059946.99        0.17            589303.69
    LinearRegression                             0.60       0.60 1059946.99        0.22            589303.69
    SGDRegressor                                 0.59       0.59 1064942.94        0.32            568783.85
    GammaRegressor                               0.56       0.57 1102941.88        0.19            497178.65
    ElasticNet                                   0.56       0.56 1112092.69        0.90            573414.56
    MLPRegressor                                 0.55       0.55 1119725.78       90.25            602807.65
    HuberRegressor                               0.54       0.54 1131176.81        0.96            506417.82
    PassiveAggressiveRegressor                   0.53       0.53 1148455.45        4.35            504893.12
    OrthogonalMatchingPursuitCV                  0.52       0.52 1156360.30        0.37            660319.66
    GeneralizedLinearRegressor                   0.52       0.52 1162533.40        0.31            591226.81
    TweedieRegressor                             0.52       0.52 1162533.40        0.16            591226.81
    OrthogonalMatchingPursuit                    0.50       0.50 1187462.25        0.14            673650.67
    RANSACRegressor                              0.28       0.29 1414399.98        0.66            853828.95
    ElasticNetCV                                 0.00       0.01 1668582.32        0.66           1039835.27
    DummyRegressor                              -0.00      -0.00 1672841.10        0.12           1043972.62
    NuSVR                                       -0.04      -0.03 1701501.07      314.63            954898.86
    SVR                                         -0.10      -0.10 1754668.80      280.21            933696.23
    AdaBoostRegressor                           -0.46      -0.46 2018065.70        6.36           1746968.80
    LinearSVR                                   -0.62      -0.62 2127154.00        0.20           1318233.12

#### Результат прогона LazyPredict для логарифмированной целевой переменной:

                                   Adjusted R-Squared  R-Squared  RMSE  Time Taken  mean_absolute_error
    Model                                                                                              
    ExtraTreesRegressor                          0.94       0.94  0.24       37.05                 0.17
    RandomForestRegressor                        0.94       0.94  0.24       49.53                 0.16
    XGBRegressor                                 0.94       0.94  0.25        2.17                 0.17
    BaggingRegressor                             0.93       0.93  0.26        5.16                 0.17
    HistGradientBoostingRegressor                0.93       0.93  0.26        3.41                 0.18
    LGBMRegressor                                0.93       0.93  0.26        0.57                 0.18
    SVR                                          0.92       0.92  0.27     1757.42                 0.19
    NuSVR                                        0.92       0.92  0.27     2241.37                 0.19
    MLPRegressor                                 0.91       0.91  0.30       59.78                 0.22
    GradientBoostingRegressor                    0.91       0.91  0.30       12.53                 0.22
    DecisionTreeRegressor                        0.87       0.87  0.35        0.90                 0.24
    ExtraTreeRegressor                           0.87       0.87  0.36        0.48                 0.25
    KNeighborsRegressor                          0.85       0.85  0.39       72.60                 0.28
    ElasticNetCV                                 0.83       0.83  0.40        1.21                 0.29
    BayesianRidge                                0.83       0.83  0.40        0.19                 0.29
    RidgeCV                                      0.83       0.83  0.40        0.23                 0.29
    LassoLarsCV                                  0.83       0.83  0.40        0.60                 0.29
    LarsCV                                       0.83       0.83  0.40        0.64                 0.29
    Ridge                                        0.83       0.83  0.40        0.14                 0.29
    TransformedTargetRegressor                   0.83       0.83  0.40        0.16                 0.29
    LassoLarsIC                                  0.83       0.83  0.40        0.20                 0.29
    LinearRegression                             0.83       0.83  0.40        0.18                 0.29
    Lars                                         0.83       0.83  0.40        0.44                 0.29
    LassoCV                                      0.83       0.83  0.40        1.50                 0.29
    SGDRegressor                                 0.83       0.83  0.40        0.20                 0.29
    PoissonRegressor                             0.83       0.83  0.41        0.21                 0.29
    AdaBoostRegressor                            0.83       0.83  0.41        6.59                 0.31
    HuberRegressor                               0.83       0.83  0.41        1.02                 0.28
    LinearSVR                                    0.83       0.83  0.41       15.51                 0.28
    OrthogonalMatchingPursuitCV                  0.79       0.79  0.45        0.36                 0.32
    OrthogonalMatchingPursuit                    0.76       0.76  0.48        0.14                 0.34
    RANSACRegressor                              0.75       0.75  0.49        0.63                 0.36
    GeneralizedLinearRegressor                   0.75       0.75  0.49        0.30                 0.37
    TweedieRegressor                             0.75       0.75  0.49        0.13                 0.37
    GammaRegressor                               0.75       0.75  0.49        0.15                 0.37
    PassiveAggressiveRegressor                   0.75       0.75  0.50        0.24                 0.36
    ElasticNet                                   0.25       0.26  0.85        0.15                 0.68
    LassoLars                                   -0.00      -0.00  0.99        0.17                 0.79
    DummyRegressor                              -0.00      -0.00  0.99        0.11                 0.79
    Lasso                                       -0.00      -0.00  0.99        0.18                 0.79

Итак, после прогона LazyPredict с логарифмированной и не логарифмированной целевой переменной у нас есть алгоритмы "победители":

* ExtraTreesRegressor
* RandomForestRegressor
* XGBRegressor
* BaggingRegressor

С  ними и будем в дальнейшем работать.

В дальнейшем будем использовать логарифмированный таргет (по нашей метрике он дает более "красивые" результаты, а процесс моделирования идет чуть быстрее).  Для соревнования на Kaggle достаточно будет проэкспонировать предсказание). 
Результаты работы алгоритмов будем смотреть с точностью до 3 знаков после запятой.

Так-же после прогона "одиночных" моделей попробуем стакинг для алгоритмов, которые покажут наилучший  результат.

####  3.3.1 ExtraTreesRegressor

In [33]:
# Запустим модель с дефотными параметрами:

etr = ExtraTreesRegressor(random_state=RANDOM_SEED)

etr.fit(X_train, y_train)

predict = etr.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.239%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.239%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 19.217%

In [ ]:
# ВНИМАНИЕ!!! ОЧЕНЬ ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Теперь попробуем подобрать гиперпараметры для нашей модели:

#random_grid = {'n_estimators': [int(x) for x in np.linspace(start = 100, stop = 400, num = 4)],
#               'max_features': ['auto', 'sqrt','log2'],
#               'max_depth': [int(x) for x in np.linspace(5, 15, num = 6)] + [None],
#               'min_samples_split': [2, 5, 10],
#               'min_samples_leaf': [1, 2, 4],
#               'bootstrap': [True, False]}

#etr = ExtraTreesRegressor(random_state = RANDOM_SEED)
#etr_random = RandomizedSearchCV(estimator = etr, param_distributions = random_grid,
#                                n_iter = 100, cv = 5, verbose=2,
#                                random_state=RANDOM_SEED, n_jobs = -1)

#etr_random.fit(X_train, y_train)

#display(etr_random.best_params_)

    Для логарифмированной переменной получившиеся лучшие параметры:
    {'n_estimators': 300,
    'min_samples_split': 2,
    'min_samples_leaf': 1,
    'max_features': 'auto',
    'max_depth': None,
    'bootstrap': True}
    
    Для НЕ логарифмированной переменной получившиеся лучшие параметры:
    {'n_estimators': 300,
     'min_samples_split': 5,
     'min_samples_leaf': 2,
     'max_features': 'auto',
     'max_depth': 15,
     'bootstrap': False}  

In [39]:
# Запустим модель с подобранными гиперпараметрами:

etr = ExtraTreesRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1,
                          max_features='auto', max_depth=None, bootstrap=True,
                          random_state=RANDOM_SEED)

etr.fit(X_train, y_train)

predict = etr.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.248%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.248%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 20.867%

In [ ]:
# Почему-то подобранными гиперпараметрами результаты (по нашей метрике) стали хуже.
# При подготовке submission надо будет попробовать оба варианта.

####  3.3.2 RandomForestRegressor

In [40]:
# Запустим модель с дефотными параметрами:

rf = RandomForestRegressor(random_state = RANDOM_SEED)

rf.fit(X_train, y_train)

predict = rf.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.235%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.235%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 19.267%

In [ ]:
# ВНИМАНИЕ!!! ОЧЕНЬ ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Теперь попробуем подобрать гиперпараметры для нашей модели:

#random_grid = {'n_estimators': [int(x) for x in np.linspace(start=100, stop=400, num=4)],
#               'max_features': ['auto', 'sqrt', 'log2'],
#               'max_depth': [int(x) for x in np.linspace(5, 15, num=6)] + [None],
#               'min_samples_split': [2, 5, 10],
#               'min_samples_leaf': [1, 2, 4],
#               'bootstrap': [True, False]}

#rfr = RandomForestRegressor(random_state=RANDOM_SEED)
#rfr_random = RandomizedSearchCV(estimator=rfr, param_distributions=random_grid,
#                                n_iter=100, cv=5, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)

#rfr_random.fit(X_train, np.log(y_train))

#display(rfr_random.best_params_)

    Для логарифмированной переменной получившиеся лучшие параметры:
    {'n_estimators': 300,
     'min_samples_split': 2,
     'min_samples_leaf': 1,
     'max_features': 'auto',
     'max_depth': None,
     'bootstrap': True}

    Для НЕ логарифмированной переменной получившиеся лучшие параметры:
    {'n_estimators': 300,
     'min_samples_split': 2,
     'min_samples_leaf': 1,
     'max_features': 'auto',
     'max_depth': None,
     'bootstrap': True}   

In [44]:
# Запустим модель с подобранными гиперпараметрами:

rf = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1,
                          max_features='auto', max_depth=None, bootstrap=True,
                          random_state=RANDOM_SEED)

rf.fit(X_train, y_train)

predict = rf.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.228%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.228%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 19.206%

In [46]:
# С подобранными гиперпараметрами точность по нашей метрике слегка улучшилась.
# В дальнейшем можно использовать модель именно с ними.

#### 3.3.3 XGBRegressor

In [49]:
# Запустим модель с дефотными параметрами:

xgb_reg = XGBRegressor(random_state=RANDOM_SEED)

xgb_reg.fit(X_train, y_train)

predict = xgb_reg.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.280%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.280%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 21.563%

In [ ]:
# ВНИМАНИЕ!!! ОЧЕНЬ ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Теперь попробуем подобрать гиперпараметры для нашей модели:

#random_grid = {'colsample_bytree':  [0.5, 1],
#               'learning_rate': [x for x in np.linspace(start=0.01, stop=0.05, num=5)],
#               'max_depth': [int(x) for x in np.linspace(5, 40, num=6)],
#               'objective': ['reg:squarederror', 'reg:linear']}

#xgb_reg = XGBRegressor(random_state=RANDOM_SEED)
#xgb_random = RandomizedSearchCV(estimator=xgb_reg, param_distributions=random_grid,
#                                n_iter=100, cv=5, verbose=2, random_state=RANDOM_SEED, n_jobs=-1)

#xgb_random.fit(X_train, np.log(y_train))

#display(xgb_random.best_params_)

    Для логарифмированной переменной получившиеся лучшие параметры:
    {'objective': 'reg:squarederror',
     'max_depth': 12,
     'learning_rate': 0.05,
     'colsample_bytree': 1}

    Для НЕ логарифмированной переменной получившиеся лучшие параметры:
    {'objective': 'reg:squarederror',
     'max_depth': 12,
     'learning_rate': 0.05,
     'colsample_bytree': 1}

In [59]:
# Запустим модель с подобранными гиперпараметрами:

xgb_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=1,
                          learning_rate=0.05, max_depth=12, alpha=1,
                          n_estimators=1000, random_state=RANDOM_SEED)

xgb_reg.fit(X_train, y_train)

predict = xgb_reg.predict(X_test)

print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.189%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.189%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 18.084%

In [ ]:
# С подобранными гиперпараметрами точность по нашей метрике сильно улучшилась.
# В дальнейшем можно использовать модель именно с ними.

#### 3.3.4 BaggingRegressor с ExtraTreesRegressor

In [16]:
# Используем модель с параметрами полученными ранее:

etr = ExtraTreesRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1,
                          max_features='auto', max_depth=None, bootstrap=True,
                          random_state=RANDOM_SEED)

In [17]:
# ВНИМАНИЕ!!! ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Запустим бэггинг с дефотными параметрами:

#bagging_etr = BaggingRegressor(etr, n_jobs=-1, random_state=RANDOM_SEED)

#bagging_etr.fit(X_train, y_train)

#predict = bagging_etr.predict(X_test)

#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.281%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.281%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 21.438%

In [18]:
# ВНИМАНИЕ!!! ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Попробуем увеличить число эстиминаторов:

#bagging_etr = BaggingRegressor(etr, n_estimators=5, n_jobs=-1, random_state=RANDOM_SEED)

#bagging_etr.fit(X_train, y_train)

#predict = bagging_etr.predict(X_test)

#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.286%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.286%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 21.403%

#### 3.3.5 BaggingRegressor с RandomForestRegressor

In [20]:
# Используем модель с параметрами полученными ранее:

rf = RandomForestRegressor(n_estimators=300, min_samples_split=2, min_samples_leaf=1,
                          max_features='auto', max_depth=None, bootstrap=True,
                          random_state=RANDOM_SEED)

In [21]:
# ВНИМАНИЕ!!! ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Запустим бэггинг с дефотными параметрами:

#bagging_rf = BaggingRegressor(rf, n_jobs=-1, random_state=RANDOM_SEED)

#bagging_rf.fit(X_train, y_train)

#predict = bagging_rf.predict(X_test)

#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.246%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.246%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 19.801%

In [22]:
# ВНИМАНИЕ!!! ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Попробуем увеличить число эстиминаторов:

#bagging_rf = BaggingRegressor(rf, n_estimators=5, n_jobs=-1, random_state=RANDOM_SEED)

#bagging_rf.fit(X_train, y_train)

#predict = bagging_rf.predict(X_test)

#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.249%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.249%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 19.768%

#### 3.3.6 BaggingRegressor с XGBRegressor

In [24]:
# Используем модель с параметрами полученными ранее:

xgb_reg = XGBRegressor(objective='reg:squarederror', colsample_bytree=1,
                          learning_rate=0.05, max_depth=12, alpha=1,
                          n_estimators=1000, random_state=RANDOM_SEED)

In [25]:
# ВНИМАНИЕ!!! ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Запустим бэггинг с дефотными параметрами:

#bagging_xgb = BaggingRegressor(xgb_reg , n_jobs=-1, random_state=RANDOM_SEED)

#bagging_xgb.fit(X_train, y_train)

#predict = bagging_xgb.predict(X_test)

#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.184%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.184%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 18.391%

In [26]:
# ВНИМАНИЕ!!! ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Попробуем увеличить число эстиминаторов:

#bagging_xgb = BaggingRegressor(xgb_reg, n_estimators=5, n_jobs=-1, random_state=RANDOM_SEED)

#bagging_xgb.fit(X_train, y_train)

#predict = bagging_xgb.predict(X_test)

#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.191%


    Для логарифмированной переменной: 
    Точность модели по метрике MAPE: 1.191%

    Для НЕ логарифмированной переменной:
    Точность модели по метрике MAPE: 18.485%

   Итак, после прогона 3х лучших (по результатам LazyRegressor) моделей, подбора гиперпараметров и бэггинга  с ними для логарифмированной и не логарифмированной целевой переменной можно сделать следующие выводы:
   
   1. Лучшей моделью без бэггинга в обоих случаях является XGBRegressor с подобранными гиперпараметрами.
   2. Бэггинг со всеми алгоритмами показал результат хуже чем "чистый" алгоритм.
   3. Увеличение числа эстиминаторов лишь в случае с использованием XGBRegressor улучшило результат. В двух других случаях результат становился хуже.
   4. "Серебряным призером" стал RandomForestRegressor с подобранными гиперпараметрами.
   
Для лучших моделей можно попробовать применить стекинг.

### 3.4 Стекинг

In [25]:
# ВНИМАНИЕ!!! ОЧЕНЬ ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Для стекинга возьмем модели "победители", полученные на прошлом этапе: XGBRegressor и  
# RandomForestRegressor, в обоих случаях используем наилучшие подобранные гиперпараметры.
# Проверять будем опять с логарифмированной и не логарифмированной целевой переменной.
# Финальным алгоритмом возьмем LinearRegression.

#estimators = [('rf',RandomForestRegressor(n_estimators=300, min_samples_split=2,
#                                          min_samples_leaf=1, max_features='auto',
#                                          max_depth=None, bootstrap=True,
#                                          random_state=RANDOM_SEED)),
#              ('xgb_reg',XGBRegressor(objective='reg:squarederror', colsample_bytree=1,
#                                      learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1,
#                                      n_estimators=1000, random_state=RANDOM_SEED))]

#st_reg = StackingRegressor(estimators=estimators,final_estimator=LinearRegression(),n_jobs=-1)

#st_reg.fit(X_train, y_train)

#predict = st_reg.predict(X_test)

#print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.3f}%")

Точность модели по метрике MAPE: 1.172%


    Для логарифмированной переменной: 
    Точность наивной модели по метрике MAPE: 1.172%

    Для НЕ логарифмированной переменной:
    Точность наивной модели по метрике MAPE: 17.919%

### 3.5 Выбор модели

   Итак, после прогона всех моделей (наивной, CatBoostRegressor, ExtraTreesRegressor, RandomForestRegressor, XGBRegressor, BaggingRegressor и стекинга)  можно подводить результаты:
   
   1. Лучше всего себя показал стекинг на моделях "победителях" из LazyPredict (XGBRegressor и  RandomForestRegressor) с линейной регрессией в качестве  решающего алгоритма.
   2. Из одиночных  моделей лучше всех оказались CatBoostRegressor и XGBRegressor.
   3. Модели, запущенные с логарифмированным таргетом, по нашей метрике  показывают результат  на порядок лучше чем с НЕ логарифмированной целевой переменной  (когда будет время, надо будет разобраться почему,  ведь наша метрика безразмерная).
   4. Для соревнования на Kaggle  я решил подготовить оба submisssion'а (с логарифмированным и не логарифмированным таргетом),  что-бы посмотреть какой даст лучший результат.

## 4. Подготовка submission

Теперь осталось лишь запустить наилучшую модель и подготовить submission-файл для соревнования.

Для логарифмированной целевой переменной надо не забыть экспонировать предсказание.

Так-же необходимо ввести поправочный коэффициент, для корректировки роста стоимости б/у автомобилей. Согласно сайту автостата (https://www.autostat.ru/infographics/48987/) рост цен в 2021 составил +28% год к году от 2020.
Эти  данные будем использовать для корректировки предсказания.

In [19]:
# Посмотрим на файл с образцом submission:

submission = pd.read_csv(PATH_to_file + 'sample_submission.csv')

display(submission)

,sell_id,price
0,1100575026,0
1,1100549428,0
2,1100658222,0
3,1100937408,0
4,1101037972,0
...,...,...
34681,1101369263,0
34682,1101369581,0
34683,1101364889,0
34684,1101362518,0


In [22]:
# Рассчитаем поправочный коэффициент  для предсказания:

price_coeff = np.round(1/1.28, 3)

In [33]:
# Итак, нам надо сделать предсказание и заполнить колонку price.
# Сделаем это дважды для логарифмированного и не логарифмированного таргета.

### 4.1 Логарифмированный таргет

In [34]:
# Подготовим данные для обучения модели, переразобьем тренировочные данные с
# теми-же параметрами что и раньше:

y = df_train.price

X = df_train.drop(columns = 'price', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    shuffle=True, 
                                                    random_state=RANDOM_SEED)

In [ ]:
# ВНИМАНИЕ!!! ОЧЕНЬ ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Переобучим наилучшую модель:

#estimators = [('rf',RandomForestRegressor(n_estimators=300, min_samples_split=2,
#                                          min_samples_leaf=1, max_features='auto',
#                                          max_depth=None, bootstrap=True,
#                                          random_state=RANDOM_SEED)),
#              ('xgb_reg',XGBRegressor(objective='reg:squarederror', colsample_bytree=1,
#                                      learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1,
#                                      n_estimators=1000, random_state=RANDOM_SEED))]

#st_reg = StackingRegressor(estimators=estimators,final_estimator=LinearRegression(),n_jobs=-1)

#st_reg.fit(X_train, y_train)

In [37]:
# Сделаем предсказание для тестовых данных:

sub_pred_log = st_reg.predict(df_test)

In [64]:
# Теперь надо проэкспонировать предсказание:

price_pred = np.exp(sub_pred_log)

In [65]:
# Помножим предсказание на поправочный коэффициент:

price_pred = price_pred*price_coeff

In [66]:
# Округлим предсказание до 1000, т.к. почти всегда цены на автомобили выставляют "круглыми":

price_pred = np.round(price_pred/1000)*1000

In [ ]:
# Сохраним наше предсказание в отдельный датасет:

submission_log = submission.copy()

submission_log.price = price_pred

In [72]:
# Сохраним наше предсказание в отдельный файл:

submission_log.to_csv(PATH_to_file + 'submission_log.csv', index=False)

In [25]:
# Убедимся что он правильно сохранился:

pd.read_csv(PATH_to_file + 'submission_log.csv')

,sell_id,price
0,1100575026,736000.00
1,1100549428,962000.00
2,1100658222,1079000.00
3,1100937408,800000.00
4,1101037972,671000.00
...,...,...
34681,1101369263,1268000.00
34682,1101369581,2128000.00
34683,1101364889,194000.00
34684,1101362518,1078000.00


### 4.2 НЕ логарифмированный таргет

In [73]:
# Подготовим данные для обучения модели, переразобьем тренировочные данные с
# теми-же параметрами что и раньше:

y = np.exp(df_train.price)

X = df_train.drop(columns = 'price', axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    shuffle=True, 
                                                    random_state=RANDOM_SEED)

In [ ]:
# ВНИМАНИЕ!!! ОЧЕНЬ ДОЛГИЙ ПРОЦЕСС!!! НЕ ПЕРЕЗАПУСКАТЬ БЕЗ НЕОБХОДИМОСТИ!!!

# Переобучим наилучшую модель:

#estimators = [('rf',RandomForestRegressor(n_estimators=300, min_samples_split=2,
#                                          min_samples_leaf=1, max_features='auto',
#                                          max_depth=None, bootstrap=True,
#                                          random_state=RANDOM_SEED)),
#              ('xgb_reg',XGBRegressor(objective='reg:squarederror', colsample_bytree=1,
#                                      learning_rate=0.05, max_depth=12, alpha=1, n_jobs=-1,
#                                      n_estimators=1000, random_state=RANDOM_SEED))]

#st_reg = StackingRegressor(estimators=estimators,final_estimator=LinearRegression(),n_jobs=-1)

#st_reg.fit(X_train, y_train)

In [75]:
# Сделаем предсказание для тестовых данных:

sub_pred_norm = st_reg.predict(df_test)

In [76]:
# Помножим предсказание на поправочный коэффициент:

price_pred = sub_pred_norm*price_coeff

In [77]:
# Округлим предсказание до 1000, т.к. почти всегда цены на автомобили выставляют "круглыми":

price_pred = np.round(price_pred/1000)*1000

In [ ]:
# Сохраним наше предсказание в отдельный датасет:

submission_norm = submission.copy()

submission_norm.price = price_pred

In [79]:
# Сохраним наше предсказание в отдельный файл:

submission_norm.to_csv(PATH_to_file + 'submission_norm.csv', index=False)

In [23]:
# Убедимся что он правильно сохранился:

pd.read_csv(PATH_to_file + 'submission_norm.csv')

,sell_id,price
0,1100575026,708000.00
1,1100549428,971000.00
2,1100658222,1106000.00
3,1100937408,824000.00
4,1101037972,713000.00
...,...,...
34681,1101369263,1221000.00
34682,1101369581,2057000.00
34683,1101364889,211000.00
34684,1101362518,1142000.00


## 5. Выводы

   После публикации резултатов на Kaggle стало понятно, что решение полученное с логарифмированной целевой переменной лучше.
   
   Но даже с ним счет на данный момент всего 15.17236 :(
   
   При наличии времени буду и дальше пытаться улучшить свое решение (как минимум перезапущу парсинг данных, что-бы собрать информацию о времени владения автомобилем). К сожалению, что парсинг данных, что тренировка моделей занимают ОЧЕНЬ МНОГО времени и как их ускорить (кроме покупки более нового более мощного компьютера:)) я пока не представляю.